### Kombinasi 8 :
* Delete Duplicate
* Delete Null
* Outlier capping pake log
* Encoding 
* Standard scaler
* Feature selection -> K-Best

In [602]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [603]:
df = pd.read_csv("../UFC_train.csv")
df_test = pd.read_csv('../regression/New_UFC_Test_Regression_X(3).csv')
df.head()

,R_fighter,B_fighter,Referee,date,location,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,Winner
0,Joe Riggs,Joe Doerksen,Steve Mazzagatti,2004-08-21,"Las Vegas, Nevada, USA",False,Middleweight,NaN,NaN,NaN,...,0,0,0,Southpaw,182.88,177.80,185.0,26.0,21.0,Red
1,Jorge Masvidal,Al Iaquinta,Keith Peterson,2015-04-04,"Fairfax, Virginia, USA",False,Lightweight,1.156250,0.000000,0.394141,...,0,1,0,Orthodox,180.34,187.96,170.0,27.0,30.0,Blue
2,Dan Stittgen,Stephen Thompson,Josh Rosenthal,2012-02-04,"Las Vegas, Nevada, USA",False,Welterweight,NaN,NaN,NaN,...,0,0,0,Orthodox,185.42,NaN,170.0,28.0,31.0,Blue
3,Josh Koscheck,Johny Hendricks,Kevin Mulhall,2012-05-05,"East Rutherford, New Jersey, USA",False,Welterweight,0.695312,0.000000,0.783359,...,6,3,0,Orthodox,177.80,185.42,170.0,28.0,34.0,Blue
4,John Dodson,Manvel Gamburyan,James Warring,2016-04-16,"Tampa, Florida, USA",False,Bantamweight,0.500000,0.266602,0.381462,...,3,0,1,Orthodox,160.02,167.64,135.0,34.0,31.0,Red


In [604]:
# Delete duplicate
print(f"Shape before dropping duplicates : {df.shape}") # Before dropping duplicates
df.drop_duplicates(inplace=True)
print(f"Shape after dropping duplicates: {df.shape}") # After dropping duplicates

Shape before dropping duplicates : (5410, 144)
Shape after dropping duplicates: (5410, 144)


In [605]:
def check_null(df):
    col_na = df.isnull().sum().sort_values(ascending=True)
    percent = col_na / len(df)
    missing_data = pd.concat([col_na, percent], axis=1, keys=['Total', 'Percent'])

    if (missing_data[missing_data['Total'] > 0].shape[0] == 0):
        print("Tidak ditemukan missing value pada dataset")
    else:
        print(missing_data[missing_data['Total'] > 0])

In [606]:
check_null(df)

                         Total   Percent
R_Weight_lbs                 2  0.000370
R_Height_cms                 4  0.000739
B_Weight_lbs                 8  0.001479
B_Height_cms                10  0.001848
R_Stance                    27  0.004991
...                        ...       ...
B_avg_GROUND_landed       1293  0.239002
B_avg_GROUND_att          1293  0.239002
B_avg_opp_CLINCH_landed   1293  0.239002
B_avg_TD_att              1293  0.239002
B_avg_HEAD_landed         1293  0.239002

[109 rows x 2 columns]


In [607]:
# Split string and number
# Split dataframe into object type and number type
df_test_object = df_test.select_dtypes(include='object')
df_test_number = df_test.select_dtypes(include=np.number)

In [608]:
# Impute missing value using iterative imputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.tree import ExtraTreeRegressor

imputer = IterativeImputer(initial_strategy='median', random_state=42, estimator=ExtraTreeRegressor(random_state=42), max_iter=15)

df_test_imputed_number = imputer.fit_transform(df_test_number)
df_test_imputed_number = pd.DataFrame(df_test_imputed_number, columns=df_test_number.columns)

c:\Python312\Lib\site-packages\sklearn\impute\_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [609]:
df.dropna(inplace=True)

In [610]:
df_test = pd.concat([df_test_object, df_test_imputed_number], axis=1)

In [611]:
check_null(df_test)

          Total   Percent
R_Stance      2  0.002561
Referee       2  0.002561


In [612]:
# Split string and number
# Split dataframe into object type and number type
df_test_object = df_test.select_dtypes(include='object')
df_test_number = df_test.select_dtypes(include=np.number)

In [613]:
# Drop R_fighter, B_fighter, Referee, location, and date
df = df.drop(['R_fighter', 'B_fighter', 'Referee', 'location', 'date', 'Winner'], axis=1)
df_test_object = df_test_object.drop(['R_fighter', 'B_fighter', 'Referee', 'location', 'date'], axis=1)

In [614]:
check_null(df)

Tidak ditemukan missing value pada dataset


In [615]:
# Combine df_object and df_imputed_number
df_test = pd.concat([df_test_object, df_test_number], axis=1)
df.head()

,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
1,False,Lightweight,1.156250,0.000000,0.394141,0.352422,0.239219,0.011484,0.15625,0.132812,...,4,0,1,0,Orthodox,180.34,187.96,170.0,27.0,30.0
3,False,Welterweight,0.695312,0.000000,0.783359,0.185547,0.088281,0.104375,0.09375,0.062500,...,5,6,3,0,Orthodox,177.80,185.42,170.0,28.0,34.0
4,False,Bantamweight,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,1.37793,0.000244,...,2,3,0,1,Orthodox,160.02,167.64,135.0,34.0,31.0
6,False,Lightweight,0.000000,0.000000,0.470000,0.250000,0.750000,0.000000,0.00000,0.000000,...,1,1,1,1,Orthodox,180.34,193.04,155.0,32.0,30.0
7,False,Welterweight,0.500000,0.000000,0.460000,0.292500,0.125000,0.100000,0.00000,0.000000,...,2,0,0,0,Orthodox,182.88,185.42,170.0,32.0,27.0


In [616]:
check_null(df)

Tidak ditemukan missing value pada dataset


In [617]:
# Impute missing R_stance and B_stance with mode
df['R_Stance'].fillna(df['R_Stance'].mode()[0], inplace=True)
df['B_Stance'].fillna(df['B_Stance'].mode()[0], inplace=True)

df_test['R_Stance'].fillna(df_test['R_Stance'].mode()[0], inplace=True)
df_test['B_Stance'].fillna(df_test['B_Stance'].mode()[0], inplace=True)

In [618]:
check_null(df)

Tidak ditemukan missing value pada dataset


In [619]:
def check_outlier(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)

    # Menghitung RUB dan RLB.
    IQR = Q3 - Q1
    lower_limit = Q1 - 1.5*IQR
    upper_limit = Q3 + 1.5*IQR

    # Menampilkan banyaknya outlier pada atribut.
    outliers = (df < lower_limit) | (df > upper_limit)
    print ("Outlier pada tiap atribut:")
    print(outliers.sum())

    return outliers

In [620]:
# Split string and number
# Split dataframe into object type and number type
df_object = df.select_dtypes(include='object')
df_number = df.select_dtypes(include=np.number)

df_test_object = df_test.select_dtypes(include='object')
df_test_number = df_test.select_dtypes(include=np.number)

In [621]:
outliers = dict(check_outlier(df_number).sum())
print("\n\npercentage of outliers in each column:")
for key in outliers.keys():
    print(f"{key} = {outliers[key]/df_number[key].shape[0] * 100}%")

Outlier pada tiap atribut:
B_avg_KD                  67
B_avg_opp_KD             277
B_avg_SIG_STR_pct         91
B_avg_opp_SIG_STR_pct     82
B_avg_TD_pct               0
                        ... 
R_Height_cms              12
R_Reach_cms               55
R_Weight_lbs             172
B_age                     52
R_age                     12
Length: 134, dtype: int64


percentage of outliers in each column:
B_avg_KD = 1.9252873563218391%
B_avg_opp_KD = 7.959770114942528%
B_avg_SIG_STR_pct = 2.6149425287356323%
B_avg_opp_SIG_STR_pct = 2.3563218390804597%
B_avg_TD_pct = 0.0%
B_avg_opp_TD_pct = 2.442528735632184%
B_avg_SUB_ATT = 5.373563218390805%
B_avg_opp_SUB_ATT = 6.982758620689655%
B_avg_REV = 16.580459770114942%
B_avg_opp_REV = 15.057471264367816%
B_avg_SIG_STR_att = 2.1551724137931036%
B_avg_SIG_STR_landed = 2.21264367816092%
B_avg_opp_SIG_STR_att = 2.6436781609195403%
B_avg_opp_SIG_STR_landed = 2.7873563218390807%
B_avg_TOTAL_STR_att = 1.5517241379310345%
B_avg_TOTAL_STR_landed =

In [622]:
# Add a constant value to make all values positive
df_number_positive = df_number + abs(df_number.min()) + 1

df_number_log = np.log(df_number_positive)

In [623]:
df_test_number_positive = df_test_number + abs(df_test_number.min()) + 1
df_test_number_log = np.log(df_test_number_positive)

In [624]:
outliers = dict(check_outlier(df_number_log).sum())
print("\n\npercentage of outliers in each column:")
for key in outliers.keys():
    print(f"{key} = {outliers[key]/df_number_log[key].shape[0] * 100}%")

Outlier pada tiap atribut:
B_avg_KD                  21
B_avg_opp_KD             251
B_avg_SIG_STR_pct         88
B_avg_opp_SIG_STR_pct     75
B_avg_TD_pct               0
                        ... 
R_Height_cms              12
R_Reach_cms               55
R_Weight_lbs             126
B_age                     46
R_age                      8
Length: 134, dtype: int64


percentage of outliers in each column:
B_avg_KD = 0.603448275862069%
B_avg_opp_KD = 7.212643678160919%
B_avg_SIG_STR_pct = 2.528735632183908%
B_avg_opp_SIG_STR_pct = 2.1551724137931036%
B_avg_TD_pct = 0.0%
B_avg_opp_TD_pct = 0.0%
B_avg_SUB_ATT = 1.6666666666666667%
B_avg_opp_SUB_ATT = 3.8218390804597697%
B_avg_REV = 16.522988505747126%
B_avg_opp_REV = 14.913793103448276%
B_avg_SIG_STR_att = 2.557471264367816%
B_avg_SIG_STR_landed = 2.5%
B_avg_opp_SIG_STR_att = 2.7011494252873565%
B_avg_opp_SIG_STR_landed = 2.7011494252873565%
B_avg_TOTAL_STR_att = 3.2758620689655173%
B_avg_TOTAL_STR_landed = 3.0172413793103448%
B_avg_o

In [625]:
df = pd.concat([df_object, df_number_log], axis=1)
df_test = pd.concat([df_test_object, df_test_number_log], axis=1)

In [626]:
check_null(df_test)

Tidak ditemukan missing value pada dataset


In [627]:
df['gender'] = df['weight_class'].apply(lambda x: 'women' if 'women' in x.lower() else 'male')
df_test['gender'] = df_test['weight_class'].apply(lambda x: 'women' if 'women' in x.lower() else 'male')

In [628]:
df.head()

,weight_class,B_Stance,R_Stance,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,gender
1,Lightweight,Orthodox,Orthodox,0.768371,0.000000,0.332278,0.301897,0.214481,0.011419,0.145182,...,1.609438,0.000000,0.693147,0.000000,5.817944,5.832938,5.655992,3.871201,3.912023,male
3,Welterweight,Southpaw,Orthodox,0.527867,0.000000,0.578499,0.170204,0.084600,0.099280,0.089612,...,1.791759,1.945910,1.386294,0.000000,5.810362,5.825469,5.655992,3.891820,3.988984,male
4,Bantamweight,Orthodox,Orthodox,0.405465,0.236337,0.323143,0.376076,0.357405,0.384970,0.866230,...,1.098612,1.386294,0.000000,0.693147,5.755616,5.771566,5.525453,4.007333,3.931826,male
6,Lightweight,Orthodox,Orthodox,0.000000,0.000000,0.385262,0.223144,0.559616,0.000000,0.000000,...,0.693147,0.693147,0.693147,0.693147,5.817944,5.847710,5.602119,3.970292,3.912023,male
7,Welterweight,Orthodox,Orthodox,0.405465,0.000000,0.378436,0.256578,0.117783,0.095310,0.000000,...,1.098612,0.000000,0.000000,0.000000,5.825469,5.825469,5.655992,3.970292,3.850148,male


In [629]:
df['weight_class'] = df['weight_class'].str.replace('Women', '')
df_test['weight_class'] = df_test['weight_class'].str.replace('Women', '')

In [630]:
# Get all weight_class values
weight_class = df['weight_class'].unique()
weight_class

array(['Lightweight', 'Welterweight', 'Bantamweight', 'Flyweight',
       'LightHeavyweight', 'Strawweight', 'Featherweight', 'Middleweight',
       'Heavyweight', 'CatchWeight'], dtype=object)

In [631]:
# Create a dictionary to map weight_class values to numbers
weight_class_dict = {
    'CatchWeight' : 0,
    'Strawweight' : 1,
    'Flyweight' : 2,
    'Bantamweight' : 3,
    'Featherweight' : 4,
    'Lightweight' : 5,
    'Welterweight' : 6,
    'Middleweight' : 7,
    'LightHeavyweight' : 8,
    'Heavyweight' : 9,
    'OpenWeight' : 10,
}

gender_dict = {
    'male' : 1,
    'women' : 0
}

In [632]:
# Map each weight_class value to the correct number
df['weight_class'] = df['weight_class'].map(weight_class_dict)
df_test['weight_class'] = df_test['weight_class'].map(weight_class_dict)

In [633]:
# Map each gender value to the correct number
df['gender'] = df['gender'].map(gender_dict)
df_test['gender'] = df_test['gender'].map(gender_dict)

In [634]:
df.head()

,weight_class,B_Stance,R_Stance,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,gender
1,5,Orthodox,Orthodox,0.768371,0.000000,0.332278,0.301897,0.214481,0.011419,0.145182,...,1.609438,0.000000,0.693147,0.000000,5.817944,5.832938,5.655992,3.871201,3.912023,1
3,6,Southpaw,Orthodox,0.527867,0.000000,0.578499,0.170204,0.084600,0.099280,0.089612,...,1.791759,1.945910,1.386294,0.000000,5.810362,5.825469,5.655992,3.891820,3.988984,1
4,3,Orthodox,Orthodox,0.405465,0.236337,0.323143,0.376076,0.357405,0.384970,0.866230,...,1.098612,1.386294,0.000000,0.693147,5.755616,5.771566,5.525453,4.007333,3.931826,1
6,5,Orthodox,Orthodox,0.000000,0.000000,0.385262,0.223144,0.559616,0.000000,0.000000,...,0.693147,0.693147,0.693147,0.693147,5.817944,5.847710,5.602119,3.970292,3.912023,1
7,6,Orthodox,Orthodox,0.405465,0.000000,0.378436,0.256578,0.117783,0.095310,0.000000,...,1.098612,0.000000,0.000000,0.000000,5.825469,5.825469,5.655992,3.970292,3.850148,1


In [635]:
# Get the remaining object columns
df_object = df.select_dtypes(include='object')

# Perform one-hot encoding
df_encoded = pd.get_dummies(df_object)

# Combine the encoded columns with the original dataframe
df = pd.concat([df, df_encoded], axis=1)

# Drop the original object columns
df.drop(df_object.columns, axis=1, inplace=True)

df.head()

,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,B_avg_REV,...,R_age,gender,B_Stance_Open Stance,B_Stance_Orthodox,B_Stance_Southpaw,B_Stance_Switch,R_Stance_Open Stance,R_Stance_Orthodox,R_Stance_Southpaw,R_Stance_Switch
1,5,0.768371,0.000000,0.332278,0.301897,0.214481,0.011419,0.145182,0.124703,0.000000,...,3.912023,1,False,True,False,False,False,True,False,False
3,6,0.527867,0.000000,0.578499,0.170204,0.084600,0.099280,0.089612,0.060625,0.000000,...,3.988984,1,False,False,True,False,False,True,False,False
4,3,0.405465,0.236337,0.323143,0.376076,0.357405,0.384970,0.866230,0.000244,0.117783,...,3.931826,1,False,True,False,False,False,True,False,False
6,5,0.000000,0.000000,0.385262,0.223144,0.559616,0.000000,0.000000,0.000000,0.000000,...,3.912023,1,False,True,False,False,False,True,False,False
7,6,0.405465,0.000000,0.378436,0.256578,0.117783,0.095310,0.000000,0.000000,0.000000,...,3.850148,1,False,True,False,False,False,True,False,False


In [636]:
# Get the remaining object columns
df_test_object = df_test.select_dtypes(include='object')

# Perform one-hot encoding
df_test_encoded = pd.get_dummies(df_test_object)

# Combine the encoded columns with the original dataframe
df_test = pd.concat([df_test, df_test_encoded], axis=1)

# Drop the original object columns
df_test.drop(df_test_object.columns, axis=1, inplace=True)

df_test.head()

,weight_class,id,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,...,B_age,R_age,gender,B_Stance_Orthodox,B_Stance_Southpaw,B_Stance_Switch,R_Stance_Open Stance,R_Stance_Orthodox,R_Stance_Southpaw,R_Stance_Switch
0,1,0.000000,0.405465,0.000000,0.500775,0.392042,0.000000,0.148420,0.000000,0.000000,...,3.951244,3.761200,0,True,False,False,False,True,False,False
1,3,0.693147,0.000000,0.000000,0.521766,0.398776,0.559616,0.693147,0.405465,0.405465,...,3.951244,3.850148,0,True,False,False,False,True,False,False
2,7,1.098612,0.450380,0.363118,0.532600,0.401902,0.004437,0.153746,0.004324,0.226896,...,4.204693,4.060443,1,True,False,False,False,False,True,False
3,2,1.386294,0.405465,0.000000,0.441476,0.343590,0.223144,0.223144,1.098612,0.000000,...,3.871201,3.891820,1,False,True,False,False,True,False,False
4,3,1.609438,0.000000,0.000000,0.239017,0.402126,0.285179,0.165514,0.000000,0.000000,...,3.891820,3.850148,1,True,False,False,False,False,True,False


In [637]:
df.to_csv('../Without Feature Selection/UFC_kombinasi8.csv', index=False)

In [638]:
from sklearn.feature_selection import SelectKBest, f_regression
# Perform feature selection using SelectKBest
selector = SelectKBest(k=25, score_func=f_regression)
X = df.drop('B_Reach_cms', axis=1) 
y = df['B_Reach_cms'] 
X_selected = selector.fit_transform(X, y)

# Get the selected feature names
selected_feature_names = X.columns[selector.get_support()]
print(selected_feature_names)
# Create a new dataframe with only the selected features
df_selected = pd.DataFrame(X_selected, columns=selected_feature_names)
df_selected.head()

Index(['weight_class', 'B_avg_SIG_STR_att', 'B_avg_opp_SIG_STR_att',
       'B_avg_opp_SIG_STR_landed', 'B_avg_TOTAL_STR_att',
       'B_avg_opp_TOTAL_STR_att', 'B_avg_opp_TOTAL_STR_landed',
       'B_avg_HEAD_att', 'B_avg_opp_HEAD_att', 'B_avg_opp_BODY_att',
       'B_avg_opp_BODY_landed', 'B_avg_DISTANCE_att', 'B_avg_opp_DISTANCE_att',
       'B_avg_opp_DISTANCE_landed', 'B_win_by_KO/TKO', 'B_Height_cms',
       'B_Weight_lbs', 'R_avg_TOTAL_STR_att', 'R_avg_BODY_att',
       'R_avg_opp_BODY_att', 'R_win_by_KO/TKO', 'R_Height_cms', 'R_Reach_cms',
       'R_Weight_lbs', 'gender'],
      dtype='object')


,weight_class,B_avg_SIG_STR_att,B_avg_opp_SIG_STR_att,B_avg_opp_SIG_STR_landed,B_avg_TOTAL_STR_att,B_avg_opp_TOTAL_STR_att,B_avg_opp_TOTAL_STR_landed,B_avg_HEAD_att,B_avg_opp_HEAD_att,B_avg_opp_BODY_att,...,B_Height_cms,B_Weight_lbs,R_avg_TOTAL_STR_att,R_avg_BODY_att,R_avg_opp_BODY_att,R_win_by_KO/TKO,R_Height_cms,R_Reach_cms,R_Weight_lbs,gender
0,5.0,4.930588,4.515827,3.474970,4.957311,4.536707,3.528656,4.773329,4.433325,1.601595,...,5.802722,5.602119,5.062793,3.120436,2.922826,0.000000,5.817944,5.832938,5.655992,1.0
1,6.0,3.694333,3.656526,2.742854,3.960813,3.980557,3.390200,3.445762,3.393617,2.035522,...,5.795024,5.707110,4.586301,2.217149,2.257051,1.945910,5.810362,5.825469,5.655992,1.0
2,3.0,4.324430,4.551767,3.770761,4.636382,4.847017,4.281648,4.079129,4.346158,2.435210,...,5.763622,5.525453,4.917411,3.265580,3.221996,1.386294,5.755616,5.771566,5.525453,1.0
3,5.0,3.891820,4.166665,2.833213,4.653960,4.283587,3.218876,3.663562,3.828641,2.564949,...,5.787265,5.564520,4.524502,1.963610,2.772589,0.693147,5.817944,5.847710,5.602119,1.0
4,6.0,4.738389,4.412798,3.442019,4.848116,4.545951,3.695110,4.418841,4.252060,2.140066,...,5.810362,5.655992,5.072044,2.788093,2.063693,0.000000,5.825469,5.825469,5.655992,1.0


In [639]:
y = df['B_Reach_cms'].reset_index(drop=True)

In [640]:
y

0       5.802722
1       5.795024
2       5.779446
3       5.771566
4       5.840351
          ...   
3475    5.755616
3476    5.840351
3477    5.810362
3478    5.840351
3479    5.855015
Name: B_Reach_cms, Length: 3480, dtype: float64

In [641]:
df_selected = pd.concat([df_selected, y], axis=1)

In [642]:
df_selected.to_csv('../Punya Andi/UFC_kombinasi8.csv', index=False)

In [643]:
df_test_id = df_test['id']
df_test = df_test.drop(columns=['id'])
df_test = df_test.reindex(columns=df_selected.columns, fill_value=0)
df_test = pd.concat([df_test_id, df_test], axis=1)

In [644]:
df_test.to_csv('../regression_kaggle/UFC_kombinasi8.csv', index=False)

In [645]:
y_has_missing_values = y.isnull().any()
print(f"Does y have any missing values? {y_has_missing_values}")


Does y have any missing values? False


In [646]:
len(y)

3480

In [647]:
df_selected.shape

(3480, 26)

In [648]:
check_null(df_selected)

Tidak ditemukan missing value pada dataset
